In [1]:
import requests
from bs4 import BeautifulSoup
import re
from time import sleep
import json

In [2]:
# Dữ liệu nhà đất bán
try:
    with open('tempdata_ban.json', 'r', encoding='utf-8') as f:
        listdata_ban = json.loads(f.read())
        f.close()
except:
    listdata_ban = {}
try:
    with open('currentPage_ban.txt', 'r') as f:
        currentPage = int(f.read())+1
        f.close()
except:
    currentPage = 1
try:
    with open("projectLinks_ban.txt", "r") as f:
        projectLinks_ban = [i for i in f.read().split('\n')]
        f.close()
except:
    projectLinks_ban = []
try:
    with open("postLinks_ban.txt", "r") as f:
        postLinks_ban = [i for i in f.read().split('\n')]
        f.close()
except:
    postLinks_ban = []

with open("projectLinks_ban.txt", "w") as f:
    f.write("")
    f.close()
with open("projectLinks_ban.txt", "a") as f:
    for i in projectLinks_ban:
        f.write(i+'\n')
    f.close()
    
current = len(listdata_ban)+1

for page in range(currentPage, 600, 1):
    print('page', page)
    response = requests.get("https://houseviet.vn/nha-dat-ban/p{}".format(page))
    posts = BeautifulSoup(response.content).find_all('div', {'class', re.compile('item-list')})
    # posts = BeautifulSoup(response.content, "html.parser").find_all('div', {'class', 'item-list fnowrap'}) or BeautifulSoup(response.content, "html.parser").find_all('div', {'class', 'item-list fnowrap'})
    for post in posts:
        Link_Title_Img = post.find('div', {'class', 'img-item'})
        link = Link_Title_Img.find('a').get('href')
        title = Link_Title_Img.find('a').get('title')
        img = Link_Title_Img.find('img', {'class', 'lazy img-object-cover lazyload blur-up'}).get('src')
        itemContent = post.find('div', {'class', 'item-content'})
        time = itemContent.find('p', {'class', 'time'}).find('span', {'class', 'moment'}).get('data-time')
        price = itemContent.find('p', {'class', 'price'}).find('span', {'class', 'text'}).text
        area = itemContent.find('div', {'class', 'area'}).find('span', {'class', 'text'}).text
        Address = itemContent.find('div', {'class', 'address'}).find_all('a')
        address = Address[0].text+', '+Address[1].text or '' 
        sleep(1)
    #     print(link,title,img, time, price, area, address)
        postResponse = requests.get(link)
        postContent = BeautifulSoup(postResponse.content, "html.parser")
        moreInfo_ = postContent.find('ul', {'class', 'fnowrap fcenter short-detail'})
        with open("postLinks_ban.txt", "a") as f:
            f.write(link+'\n')
            f.close()
        if moreInfo_:
            moreInfo = moreInfo_.find_all('span')
            postId, typePost = moreInfo[0].text.strip(), moreInfo[1].text.strip()
            description = postContent.find('div', {'class', 'description'}).text.replace("Thông tin mô tả:", "")
            imgList = [img.get('data-background') for img in (postContent.find_all('div', {'class', 'swiper-slide swiper-lazy'}))] or []
            info_realestates = postContent.find('div', {'class', 'infomation'}).find('div', {'class', 'info-realestate'}).find_all('span')
            infoRealestate = {info_realestates[i].text.replace(":","").strip():info_realestates[i+1].text.strip() for i in range(0, len(info_realestates), 2)}

        #     print(postId)
        #     print(typePost)
        #     print(description)
        #     print(imgList)
        #     print(infoRealestate)
            projectInfo = postContent.find('div', {'class', 'related-project'})
            projectInfor = {}
            if projectInfo:
                projectDetails = projectInfo.find('div', {'class', 'related-project-detail'})
                projectImg_ = projectInfo.find('div', {'class', 'related-project-img'})
                projectImage = projectImg_.find('img').get('src')
                projectName = projectImg_.find('a').get('title')
                projectLink = projectImg_.find('a').get('href')
                #print(projectImage, projectName, projectLink)
                ownerInvestment, projectArea, legalDocument = [i.text.replace("Chủ đầu tư:", "").replace("Quy mô:", "").replace("Giấy tờ pháp lý:", "").replace("Xem Chi Tiết Dự Án", "").strip() for i in projectDetails.find_all('p')]
                if projectLink not in projectLinks_ban:
                    projectLinks_ban.append(projectLink)
                    with open("projectLinks_ban.txt", "a") as f:
                        f.write(projectLink+'\n')
                        f.close()
                projectInfor = {"name":projectName, "img":projectImage, "link":projectLink, "ownerInvestment":ownerInvestment, "projectArea":projectArea, "legalDocument":legalDocument}
            else:
                projectInfor = {}
            listdata_ban[postId] = {"title":title, "type":typePost, "img":img, "link":link, "description":description, "imgList":imgList, "price":price, "address":address, "area":area, "time":time, "infoRealestate":infoRealestate, "projectInfo":projectInfor}
            print(current, end = ' ')
            print(link)
            current+=1
        else:
            continue
        sleep(1)
    # lưu data và page tiếp theo sau mỗi page
    with open('tempdata_ban.json', 'w', encoding='utf-8') as f:
        json.dump(listdata_ban, f, ensure_ascii=False, indent=4)
        f.close()
    with open('currentPage_ban.txt', 'w') as f:
        f.write(str(page))
        f.close()
    sleep(1)

with open('data_ban.json', 'w', encoding='utf-8') as f:
    json.dump(listdata_ban, f, ensure_ascii=False, indent=4)
    f.close()

page 516
7685 https://houseviet.vn/ban-nha-rieng-nha-mat-pho-phuong-kien-hung/ban-nha-da-sy-kien-hung-ha-dong-55m-3-tang-chi-nhinh-4-ty-o-to-vao-nha-phan-lo-an-sinh-dinh
7686 https://houseviet.vn/ban-can-ho-chung-cu-xa-tan-trieu/can-ban-chung-cu-khu-nha-o-can-bo-vien-quan-y-103-thanh-tri-ha-noi-gia-tot
7687 https://houseviet.vn/ban-nha-rieng-nha-mat-pho-duong-thach-ban/ban-nha-to-4-thach-ban-quan-long-bien-gan-cong-vien-50m-5pn-3-ty-55-lh-0945262238
7688 https://houseviet.vn/ban-nha-rieng-nha-mat-pho-duong-nguyen-xien-1/ban-nha-ngo-300-nguyen-xien-dt-31m2-x5-tang-chi-2-8-ty-tang-full-noi-that
7689 https://houseviet.vn/ban-nha-rieng-nha-mat-pho-duong-dien-bien-phu-7/dien-bien-phu-nha-phan-lo-o-to-do-an-ninh-dinh-nhat-thu-do-75m2x4t-17-4ty
7690 https://houseviet.vn/ban-nha-rieng-nha-mat-pho-duong-my-khe-1/ban-nha-3-tang-duong-my-khe-1-gan-vo-van-kiet-phuoc-my-son-tra-gia-re
7691 https://houseviet.vn/ban-nha-rieng-nha-mat-pho-duong-khuong-trung/mua-ban-nha-pho-khuong-trung-thanh-xuan-37m-

7747 https://houseviet.vn/ban-dat-trang-trai-nghi-duong-duong-ba-thien/ban-dat-cu-chi-mat-tien-duong-nhua-dien-tich-7651m2-co-300m-tho-cu-xa-nhuan-duc-60-ty
7748 https://houseviet.vn/ban-nha-rieng-nha-mat-pho-phuong-14/1-8-ty-co-ngay-nha-14m2-trung-tam-quan-3-dau-tu-giu-tien
7749 https://houseviet.vn/ban-nha-rieng-nha-mat-pho-phuong-minh-khai/ban-nha-goc-de-40m2-4-tang-mt-3-6m-chi-3-75-ty
7750 https://houseviet.vn/ban-nha-rieng-nha-mat-pho-phuong-tan-hung/ban-nha-7-lau-tran-xuan-soan-p-tan-hung-quan-7-gan-benh-vien-tan-hung
7751 https://houseviet.vn/ban-dat-tho-cu-duong-le-duan-61/chi-16tr-m2-dau-tu-dat-nen-ven-bien-tiem-nang-du-lich-tiem-nang-tang-gia
7752 https://houseviet.vn/ban-dat-tho-cu-duong-quoc-lo-1a-90/dat-nen-so-do-kdc-dong-deo-khu-pho-thuong-mai-sam-uat-nhat-phu-yen
7753 https://houseviet.vn/ban-dat-trang-trai-nghi-duong-duong-nguyen-thi-ranh/dat-vuon-cu-chi-841m2-gia-1-ty-340tr-shr-chinh-chu-gui-ban
7754 https://houseviet.vn/ban-nha-rieng-nha-mat-pho-duong-tran-phu/vi-covi

7809 https://houseviet.vn/ban-dat-nen-du-an-kim-thanh/chi-tu-1-4-ty-den-2-ty-co-hoi-tot-cho-dau-tu-va-mua-nha-de-o-dat-nen-kim-dinh
7810 https://houseviet.vn/ban-can-ho-chung-cu-ha-dong/chung-cu-dang-song-nhat-quan-ha-dong-2021-trung-tam-hanh-chinh-moi-l-h-0904-949-511
7811 https://houseviet.vn/ban-can-ho-chung-cu-duong-tan-thoi-nhat-17/ban-can-ho-70m2-view-san-bay-tan-son-nhat
7812 https://houseviet.vn/ban-biet-thu-nha-lien-ke-hai-duong-hd/dau-tu-sieu-loi-nhuan-voi-quy-hang-dep-nhat-du-an-ecoriver-hai-duong
7813 https://houseviet.vn/ban-biet-thu-nha-lien-ke-ha-dong/lien-ke-kdt-van-phu-90m2-4-tang-mt-4-5m-noi-that-cao-cap-9-ty-8
7814 https://houseviet.vn/ban-dat-nen-du-an-xa-lac-long/ban-dat-cuc-dep-khu-do-thi-lac-long-gia-17tr-m2-dien-tich-90m2
7815 https://houseviet.vn/ban-nha-rieng-nha-mat-pho-dong-da/ban-nha-dep-pho-ton-duc-thang-dong-da-lo-goc-gan-pho-45m2x4t-mt4-6m-gia-4-2-ty
7816 https://houseviet.vn/ban-dat-nen-du-an-duong-18-22/du-an-tnr-uong-bi-gio-hang-hiem-con-sot-lai-cac-l

7870 https://houseviet.vn/ban-nha-rieng-nha-mat-pho-duong-tran-phu-4/nhinh-4-ty-nha-tran-phu-4-tang-x-41m2-ngo-thong-nguyen-khuyen
7871 https://houseviet.vn/ban-nha-rieng-nha-mat-pho-duong-nguyen-van-linh/nha-mat-tien-8m-nguyen-van-linh-quan-7-gia-chi-8-ty-65-non-son-thue-dai-han
7872 https://houseviet.vn/ban-dat-tho-cu-duong-my-canh/can-ban-8m-mat-tien-duong-canh-duong-loc-ninh-dong-hoi-gia-2-1-ty-lh-0905702473
7873 https://houseviet.vn/ban-nha-rieng-nha-mat-pho-dong-da/3-thoang-kinh-doanh-hoang-ngoc-phach-dong-da-60-3-tang-mat-tien-8m-gia-8-ty
7874 https://houseviet.vn/ban-dat-nen-du-an-khu-do-thi-century-city/dat-vang-san-bay-long-thanh-chiet-khau-khung-qua-tang-khung
7875 https://houseviet.vn/ban-nha-rieng-nha-mat-pho-hoan-kiem/ban-nha-7-tang-mat-pho-hang-hanh-hoan-kiem-dt-140m-rong-5m-no-hau
7876 https://houseviet.vn/ban-nha-rieng-nha-mat-pho-ha-dong/ban-nha-tran-phu-hai-mat-pho-38m-6-ty
7877 https://houseviet.vn/ban-nha-rieng-nha-mat-pho-cau-giay/ban-ccmn-quan-hoa-130m2-6-tang-th

7932 https://houseviet.vn/ban-dat-nen-du-an-duong-30-4-7/ban-160m2-dat-mat-tien-duong-30-4-ngang-5m-gan-nga-4-le-thanh-nghi
7933 https://houseviet.vn/ban-nha-rieng-nha-mat-pho-dong-da/nha-moi-dep-5-tang-ngo-354-truong-chinh-dt-45m2-mt-4m-gia-4-7-ty
7934 https://houseviet.vn/ban-nha-rieng-nha-mat-pho-hai-ba-trung/nha-van-ho-hbt-rat-hiem-sat-pho-35m2x4t-so-net-3-3-ty
7935 https://houseviet.vn/ban-nha-rieng-nha-mat-pho-hai-ba-trung/sieu-pham-kim-nguu-57m2-o-to-tranh-kinh-doanh-dinh
7936 https://houseviet.vn/ban-can-ho-chung-cu-duong-pham-kiet/can-ho-the-sang-residence-dang-cap-can-ho-5-duoc-so-huu-lau-dai-dau-tien-tai-trung-tam-tp-da-nang
7937 https://houseviet.vn/ban-dat-tho-cu-son-tay/dat-dep-gia-sieu-re-906m2-tai-dai-quang-son-dong-son-tay-hn-cach-tl-82-chi-350m-gia-2-x-ty
7938 https://houseviet.vn/ban-dat-trang-trai-nghi-duong-duong-602/can-ban-4500m2-dat-trong-cay-view-suoi-hoa-phu-hoa-vang
7939 https://houseviet.vn/ban-nha-rieng-nha-mat-pho-hoang-mai/linh-nam-nha-dep-kinh-doanh-dinh

...

8662 https://houseviet.vn/ban-biet-thu-nha-lien-ke-duong-tinh-lo-44a-1/can-ban-can-biet-thu-oceanami-long-hai-vung-tau-vi-tri-dep
8663 https://houseviet.vn/ban-can-ho-chung-cu-duong-ho-xuan-huong-3/ra-mat-can-ho-bien-so-huu-lau-dai-doi-dien-furama-resort-duong-ho-xuan-huong-tt-dan-trai-2-nam
8664 https://houseviet.vn/ban-dat-nen-du-an-long-bien/dat-ngoc-thuy-ngo-o-to-103m-vuong-nhu-hop-5-9-ty-lien-he-0849623333-de-xem-dat
8665 https://houseviet.vn/ban-nha-rieng-nha-mat-pho-duong-dien-bien-phu-11/nha-can-ban-350m2-trung-tam-mt-duong-dien-bien-phu-gan-san-bay-cong-vien-29-3
8666 https://houseviet.vn/ban-nha-rieng-nha-mat-pho-long-bien/ban-nha-p-bo-de-dt-84m2-view-ho-o-to-do-cua
8667 https://houseviet.vn/ban-nha-rieng-nha-mat-pho-duong-khuong-huu-dung/ban-nha-2-tang-duong-khuong-huu-dung-nam-cam-le-hoa-xuan-gia-re-chi-3-ty-790-trieu
8668 https://houseviet.vn/ban-nha-rieng-nha-mat-pho-long-bien/can-ban-gap-nha-ly-son-ha-long-bien-duong-oto-tranh-74m2-nhinh-5-ty-mot-chut
8669 https://housev

8722 https://houseviet.vn/ban-nha-rieng-nha-mat-pho-ha-dong/chinh-chu-can-ban-nguyen-trai-ha-dong-5-tang-43m2-4-2-ty-0968608331
8723 https://houseviet.vn/ban-dat-tho-cu-xa-le-chi/can-ban-gap-70m2-dat-dau-gia-chi-dong-le-chi-gia-lam-duong-rong-5m-gan-truc-de-mo-rong
8724 https://houseviet.vn/ban-nha-rieng-nha-mat-pho-bac-tu-liem/ban-dat-mat-pho-dinh-quan-91m-mt5m-gia-10ty-lh-0983388925
8725 https://houseviet.vn/ban-can-ho-chung-cu-duong-le-quang-dao/ban-cc-dep-nhat-gia-tot-nhat-my-dinh-the-matrix-one-me-tri-view-cong-vien-ck-13-htls-0-3-nam
8726 https://houseviet.vn/ban-nha-rieng-nha-mat-pho-ha-dong/ban-nha-chan-chung-cu-victoria-van-phu-ha-dong-sieu-pham-kinh-doanh-gia-8-5-ty
8727 https://houseviet.vn/ban-nha-rieng-nha-mat-pho-phuong-phu-luong/nha-4t-o-phu-luong-cuc-thoang-va-dep-o-to-do-cuc-gan-dt-34m2-ngo-thong-2-mt-gia-1-8-ty-ct
8728 https://houseviet.vn/ban-can-ho-chung-cu-hoang-mai/ban-can-ho-2pn-1-view-ho-cuc-dep-tai-trung-tam-quan-ha-noi-co-co-hoi-nhan-qua-len-den-1-8-ty
8729 ht

8782 https://houseviet.vn/ban-can-ho-chung-cu-cau-giay/ban-chung-cu-mini-tai-duong-khue-cau-giay-55m-x-5-gia-6-5-ty-dong-tien-khung
8783 https://houseviet.vn/ban-biet-thu-nha-lien-ke-ha-long/chinh-chu-ban-cat-lo-can-biet-thu-song-lap-m518-phan-khu-mallorca-du-an-sun-feria-ha-long
8784 https://houseviet.vn/ban-nha-rieng-nha-mat-pho-tay-ho/xuan-la-50m2x6t-thang-may-lo-goc-ngo-thong-kinh-doanh-dinh-gia-11-5-ty
8785 https://houseviet.vn/ban-dat-nen-du-an-duong-quoc-lo-18-5/hot-dat-nen-sieu-du-an-biet-thu-doi-thuy-san-so-do-trao-tay-xay-nha-ngay-lh-0909-133347
8786 https://houseviet.vn/ban-nha-rieng-nha-mat-pho-dong-da/ban-nha-pho-thai-thinh-dong-da-ngo-nong-thong-kd-40m-5t-4-ty
8787 https://houseviet.vn/ban-dat-nen-du-an-xa-dong-xa-van-don/chinh-chu-can-ban-lo-goc-du-an-phuong-dong-van-don-mat-duong-tra-ban
8788 https://houseviet.vn/ban-nha-rieng-nha-mat-pho-long-bien/ban-nha-3-tang-ngoc-lam-o-to-do-cua-28m-mat-tien-5m-2-ty-3-lo-goc-2-mat-thoang
8789 https://houseviet.vn/ban-dat-nen-du-an-

8842 https://houseviet.vn/ban-nha-rieng-nha-mat-pho-hai-ba-trung/truong-dinh-lo-goc-ngo-ba-gac-31m-4-tang-mt-7m-2-5-ti
8843 https://houseviet.vn/ban-dat-tho-cu-duong-nguyen-tri-phuong-28/chinh-chu-ban-400m2-dat-xay-villa-kinh-doanh-hang-an-tai-nguyen-tri-phuong-hoi-an-0989-33-45-66
8844 https://houseviet.vn/ban-nha-rieng-nha-mat-pho-hoang-mai/ban-nha-ngo-o-to-duong-hoang-mai-gia-re-3pn-dt36mx5t-mt3-1m-2-9-ty
8845 https://houseviet.vn/ban-can-ho-chung-cu-ha-dong/can-ban-nhanh-can-3-ngu-goc-co-noi-that-co-ban-du-an-hpc-lanmark-105-gia-2ty4-lh-0878911233
8846 https://houseviet.vn/ban-nha-rieng-nha-mat-pho-tay-ho/nha-cuc-dep-pho-au-co-full-noi-that-tay-ho-thang-may-70m2-nhinh-9-ty
8847 https://houseviet.vn/ban-nha-rieng-nha-mat-pho-duong-dai-lo-thang-long-1/hiem-doc-quyen-lo-shophouse-sieu-pham-vuon-nhat-vinhomes-smart-city
8848 https://houseviet.vn/ban-nha-rieng-nha-mat-pho-ba-dinh/chi-3-65-ty-co-ngay-nha-doi-can-dt-60m2-2-tang-mt-7m-co-san-rieng
8849 https://houseviet.vn/ban-nha-rieng-nh

8902 https://houseviet.vn/ban-biet-thu-nha-lien-ke-xa-cam-ha-hoi-an/casamia-calm-hoi-an-co-dang-de-dau-tu
8903 https://houseviet.vn/ban-dat-nen-du-an-duong-nguyen-van-truong-1/nen-mat-tien-duong-nguyen-van-truong-10x28-tho-cu-gia-4-2-ty-binh-thuy-can-tho
8904 https://houseviet.vn/ban-dat-nen-du-an-xa-cam-thanh-hoi-an/toi-chinh-chu-can-ban-300m2-dat-duong-be-tong-6m-cam-thanh-cach-cho-ba-le-200m
8905 https://houseviet.vn/ban-dat-nen-du-an-duong-bong-giang/ban-dat-mat-tien-bong-giang-my-khanh-dt-17m-x100m-no-hau25m-gia-5-5-tr-m2-dt-0945949909
8906 https://houseviet.vn/ban-nha-rieng-nha-mat-pho-duong-bo-de/ban-nha-bo-de-long-bien-5-5-ty-oto-thang-may-50m2-6-tang
8907 https://houseviet.vn/ban-biet-thu-nha-lien-ke-duong-an-my-3/ban-villa-view-lua-duong-15m5-trung-tam-an-my-gia-8-ty
8908 https://houseviet.vn/ban-dat-nen-du-an-cai-rang/nen-truc-chinh-lo-47m-khu-nam-long-2-huong-dong-nam-5-7x25m-tho-cu
8909 https://houseviet.vn/ban-dat-nen-du-an-duong-tinh-lo-299/yen-dung-green-park-thien-duon